In [279]:
# # Small LSTM Network to Generate Text for Alice in Wonderland

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from keras.callbacks import ModelCheckpoint
# from keras.utils import np_utils
# # load ascii text and covert to lowercase
# # filename = "wonderland.txt"
# # raw_text = open(filename).read()
# raw_text = list
# # create mapping of unique chars to integers
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# # summarize the loaded data
# n_chars = len(raw_text)
# n_vocab = len(chars)
# print ("Total Characters: ", n_chars)
# print ("Total Vocab: ", n_vocab)
# # prepare the dataset of input to output pairs encoded as integers
# seq_length = 100
# dataX = []
# dataY = []
# for i in range(0, n_chars - seq_length, 1):
#     seq_in = raw_text[i:i + seq_length]
#     seq_out = raw_text[i + seq_length]
#     dataX.append([char_to_int[char] for char in seq_in])
#     dataY.append(char_to_int[seq_out])
# n_patterns = len(dataX)
# print ("Total Patterns: ", n_patterns)
# # reshape X to be [samples, time steps, features]
# X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# # normalize
# X = X / float(n_vocab)
# # one hot encode the output variable
# y = np_utils.to_categorical(dataY)
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# # define the checkpoint
# filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]
# # fit the model
# model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [435]:
import numpy as np
import re, os, pygtrie, numbers
dimension = 30
def readFile():
    list = []
    for i in os.listdir("assignment1-data/"):
        if i.startswith("training"):
            input = open("assignment1-data/"+ i, 'r')
            list.append(input.readlines())
    return(list)
def process_line(string):
    string = re.sub(r'[^A-Za-z0-9\ \.]', "", string)
    string = re.sub('[0-9]', "0",string)
    string = string.lower()
    string = "##" + string + "#"
    return(string)
processed_lines = []
list = readFile()
for l in list:
    for line in l:
        processed_lines.append(process_line(line))
#     print('-------------------')
# print(processed_lines)
# for processed_line in processed_lines:
print(processed_lines[0])

triGram = np.zeros((30,30,30))
# print(processed_lines)
for processed_line in processed_lines:
    constructTriGram(processed_line.strip(' ')) # shift it from the for above
print('done')

##resumption of the session#
done


In [391]:
def constructTriGram(line):
    length = len(line)
    listOfGrams = []
    for i in range(length):
        c = []
        if i is 0 or i is 1:
            continue
        else:
            enterSequence(line[i-2:i+1])

In [286]:
ord('.')%97

46

In [326]:
def enterSequence(sequence):
    seq=[]
#     print(sequence)
    seq.append(ord(sequence[0])%97)
    seq.append(ord(sequence[1])%97)
    seq.append(ord(sequence[2])%97)
    seq = getascii(seq)
#     print(seq)
    triGram[seq[0]][seq[1]][seq[2]] += 1

In [434]:
def getascii(seq):
    for i in range(len(seq)):
        if seq[i] == 32:
            seq[i] = 26
        if seq[i] == 46:
            seq[i] = 27
        if seq[i] == 48:
            seq[i] = 28
        if seq[i] == 35:
            seq[i] = 29
    return seq

In [328]:
def getBigram(freq_matrix):
    bi_grams = np.zeros((dimension,dimension))
    for i in range(dimension):
        for j in range(dimension):
            bi_grams[i][j] = sum(freq_matrix[i][j])
    return(bi_grams)

In [329]:
def getUnigram(bi_grams):
    uni_grams = np.zeros((dimension))
    for i in range(dimension):
        uni_grams[i] = sum(bi_grams[i])
    return(uni_grams)

In [420]:
D = [0.70, 0.20, 0.10]
charCounts = dimension
bi_gram_count = getBigram(triGram)
uni_gram_count = getUnigram(bi_gram_count)
uni_gram = (uni_gram_count / sum(uni_gram_count))
bi_gram = np.zeros((dimension,dimension))
tri_gram = np.zeros((dimension,dimension,dimension))
for i in range(dimension):
    for j in range(dimension):
        if bi_gram_count[i][j] > 0:
            bi_gram[i][j] = ( (bi_gram_count[i][j] - D) / uni_gram_count[i] ) + (((D * charCounts) / uni_gram_count[i]) * uni_gram[j])
        else:
            bi_gram[i][j] = (((D * charCounts) / uni_gram_count[i]) * uni_gram[j])
            # print(bi_gram)
for w1 in range(dimension):
    for w2 in range(dimension):
        for w3 in range(dimension):
            if triGram[w1][w2][w3] > 0:
                tri_gram[w1][w2][w3] = ((triGram[w1][w2][w3] - D) / bi_gram_count[w1][w2]) + ((D * charCounts) / bi_gram_count[w1][w2]) * bi_gram[w2][w3]
            elif bi_gram_count[w1][w2] > 0:
                tri_gram[w1][w2][w3] = ((D * charCounts) / bi_gram_count[w1][w2]) * bi_gram[w2][w3]
            else:
                tri_gram[w1][w2][w3] = 
print(tri_gram)


[[[             inf              inf              inf ...,              inf
                inf              inf]
  [  9.82648126e-03   1.77147243e-05   1.48480420e-03 ...,   8.43003521e-04
     2.33674127e-07   7.68899150e-06]
  [  5.91657235e-03   3.84031012e-06   2.55703980e-01 ...,   2.18529714e-03
     5.06573571e-08   1.40498804e-04]
  ..., 
  [  3.55872033e-02   3.20696765e-02   7.54121858e-02 ...,   4.96844572e-03
     4.65926190e-07   3.15544533e-04]
  [             inf              inf              inf ...,              inf
                inf              inf]
  [  5.26663111e-03   8.97887848e-04   4.26067764e-02 ...,   6.40774754e-01
     2.46025640e-02   2.89848804e+00]]

 [[  2.47443768e-05   1.31644646e-02   6.34519307e-02 ...,   8.72263940e-03
     5.56470242e-08   1.35852052e-04]
  [             inf              inf              inf ...,              inf
                inf              inf]
  [  1.89330315e+00   1.22889924e-03   5.75273647e-01 ...,   9.49295084e-01
  

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars


In [ ]:
# maxi = 0
# i=0
# j=0
# k=0
# for w1 in range(38):
#     for w2 in range(38):
#         for w3 in range(38):
#             if abs(tri_gram[w1][w2][w3]) > maxi:
#                 maxi = abs(tri_gram[w1][w2][w3])
#                 i = w1
#                 j = w2
#                 k = w3
# print(maxi)
# from numpy import unravel_index
# unravel_index(triGram.argmax(), triGram.shape)
# triGram[26][19][8]              

In [412]:
# unravel_index(tri_gram[26][19].argmax(), tri_gram[26][19].shape)
tri_gram

array([[[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.48480420e-03, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   2.55703980e-01, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  3.55872033e-02,   3.20696765e-02,   7.54121858e-02, ...,
           4.96844572e-03,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   2.89848804e+00]],

       [[  0.00000000e+00,   1.31644646e-02,   6.34519307e-02, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+0

In [ ]:
# class sequence(object):
#     def __init__(self, char="", probability=0):
#         self.char = char[0]
#         self.bigram = bigram(char[1])
#         self.trigram = trigram(char[2], probability)
        
#     def printTrigram(self):
#         print(self.char + self.bigram.printChar()+ self.trigram.printChar())

In [ ]:
# class bigram(self, char = '', probablility):
#     def __init__(self, char='', probability =0):
#         self.char = [char, probability]
#         self.gram =

In [394]:
bi_gram

array([[  1.43346734e-04,   1.87918407e-02,   3.42502653e-02,
          3.30734188e-02,   4.87425055e-04,   1.07768133e-02,
          1.71923935e-02,   2.23787232e-04,   2.75960204e-02,
          1.74324081e-03,   1.86698303e-02,   1.22287842e-01,
          4.13140766e-02,   1.82647944e-01,   1.85961195e-04,
          1.77477963e-02,   2.33333729e-06,   1.10239081e-01,
          8.41098725e-02,   1.64491146e-01,   1.11134935e-02,
          2.43566011e-02,   4.34093918e-03,   1.63702650e-03,
          2.14725274e-02,   2.74477679e-05,   5.05311524e-02,
          3.22368968e-07,   7.87004990e-04],
       [  7.95154346e-02,   1.43346734e-04,   5.20714485e-04,
          3.70342411e-04,   4.19944457e-01,   2.41132258e-04,
          1.83775556e-04,   5.27465559e-04,   2.99570591e-02,
          3.84914464e-02,   5.58259895e-05,   1.28721026e-01,
          4.89374826e-03,   8.26944980e-04,   4.82013226e-02,
          3.00830050e-04,   1.36863716e-05,   3.16661696e-02,
          3.60665761e-02,

In [422]:
from random import randint
theText = []
def generateText():
    temp = 0
    first = 0
    second = 0
    theOne = 0
    for i in range(500):
        theOne = randint(0,2)
        probableChar = sorted(range(len(tri_gram[first][second])), key=lambda i: tri_gram[first][second][i], reverse=True)[:3]
        theText.append((probableChar[theOne]))
        temp = (probableChar[theOne])
        first = second
        second = temp
generateText()
text = ' '
for i in range(len(theText)):
    text = text + asciiToChars(theText[i])
print(text)

 ct intion ouldevelysent int tringer se ce cousedure ce ce cand a poll ast ing.cce tood itinger socal pands anspaseduch thavot our ansit the cesione a pose canspollemendend iting.accansithe tral ther ase an as to a sommintivandevely too but to by asenderegans ansionesiting.aabour ase tran oft it in oust truchesse and astat it that ousendere tratiodus ansionspecone comper st oner tortaken there theresidir truropminte ans trultur se to triouresitivid in on tritiour astrations ansperst truresions in


In [438]:
def asciiToChars(value):
    char =''
    if value < 26:
        char = chr(int(value+97))
    elif value == 26:
        char = ' '
    elif value == 27:
        char = '.'
    elif value == 28:
        char = '0'
    elif value == 29:
        char = '#'
    return char

In [ ]:
for i in range(28):
    for j in range(28):
        print(tri_gram[i][j].argmax())

In [ ]:
asciiToChars(27)

In [309]:
def getTrainAndTestData(corpus, dev_part):
    length = len(corpus)
    folded_data = []
    folds = 5
    fold = int(length/5)
    start = 0
    print(fold)
    for i in range(len(corpus)):
        if (i%fold == 0):
            folded_data.append(corpus[start:i])
            start = i
    print(folded_data)
    dev = folded_data[dev_part]
    train = folded_data.pop(dev_part)
    print(dev)
    print('---')
    print(train)

In [311]:
getTrainAndTestData("Here we go, I am just checking",1)

6
['', 'Here w', 'e go, ', 'I am j', 'ust ch']
Here w
---
Here w


In [317]:
from nltk.tokenize import sent_tokenize
def getStratifiedSamplingOfData(corpus, dev_part):
    sent_list = sent_tokenize(corpus)
    train_data = []
    test_data = []
    for i in range(len(sent_list)):
        if i%5 == 0:
            test_data.append(sent_list[i])
        else:
            train_data.append(sent_list[i])
    print(train_data)        
    
    

In [318]:
whichPart = 0
corpus = "this’s a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it’s your turn."
getStratifiedSamplingOfData(corpus, whichPart)

['this is sent two.', 'is this sent three?', 'sent 4 is cool!', 'Now it’s your turn.']


In [400]:
print(bi_gram)

[[  1.43346734e-04   1.87918407e-02   3.42502653e-02   3.30734188e-02
    4.87425055e-04   1.07768133e-02   1.71923935e-02   2.23787232e-04
    2.75960204e-02   1.74324081e-03   1.86698303e-02   1.22287842e-01
    4.13140766e-02   1.82647944e-01   1.85961195e-04   1.77477963e-02
    2.33333729e-06   1.10239081e-01   8.41098725e-02   1.64491146e-01
    1.11134935e-02   2.43566011e-02   4.34093918e-03   1.63702650e-03
    2.14725274e-02   2.74477679e-05   5.05311524e-02   3.22368968e-07
    7.87004990e-04]
 [  7.95154346e-02   1.43346734e-04   5.20714485e-04   3.70342411e-04
    4.19944457e-01   2.41132258e-04   1.83775556e-04   5.27465559e-04
    2.99570591e-02   3.84914464e-02   5.58259895e-05   1.28721026e-01
    4.89374826e-03   8.26944980e-04   4.82013226e-02   3.00830050e-04
    1.36863716e-05   3.16661696e-02   3.60665761e-02   5.66477721e-03
    9.03126901e-02   9.05742008e-04   1.79453544e-04   2.29606892e-05
    8.19789328e-02   3.96184442e-06   6.82154574e-03   1.89088029e-06


In [439]:
file = open("trigram_add_one.en","w")
for i in range(30):
    for j in range(30):
        for k in range(30):
            s = ""
            listSeq = []
            listSeq.append(asciiToChars(i))
            listSeq.append(asciiToChars(j))
            listSeq.append(asciiToChars(k))
            listSeq.append("\t")
            listSeq.append(str(tri_gram_add_one[i][j][k]))
            listSeq.append("\n")
            file.write(s.join(listSeq)) 
file.close()
 


In [405]:
import nltk



ngrams = nltk.trigrams("What a piece of work is man! how noble in reason! how infinite in faculty! in form and moving how express and admirable! in action how like an angel! in apprehension how like a god! the beauty of the world, the paragon of animals!")



freq_dist = nltk.FreqDist(ngrams)

kneser_ney = nltk.KneserNeyProbDist(freq_dist)

prob_sum = 0

for i in kneser_ney.samples():
    print(i)
    print(kneser_ney.prob(i))



('W', 'h', 'a')
0.25
('h', 'a', 't')
0.25
('a', 't', ' ')
0.25
('t', ' ', 'a')
0.25
(' ', 'a', ' ')
0.125
('a', ' ', 'p')
0.125
(' ', 'p', 'i')
0.125
('p', 'i', 'e')
0.25
('i', 'e', 'c')
0.25
('e', 'c', 'e')
0.25
('c', 'e', ' ')
0.25
('e', ' ', 'o')
0.03125
(' ', 'o', 'f')
0.75
('o', 'f', ' ')
0.75
('f', ' ', 'w')
0.08333333333333333
(' ', 'w', 'o')
0.625
('w', 'o', 'r')
0.625
('o', 'r', 'k')
0.08333333333333333
('r', 'k', ' ')
0.25
('k', ' ', 'i')
0.25
(' ', 'i', 's')
0.03571428571428571
('i', 's', ' ')
0.25
('s', ' ', 'm')
0.125
(' ', 'm', 'a')
0.125
('m', 'a', 'n')
0.125
('a', 'n', '!')
0.041666666666666664
('n', '!', ' ')
0.625
('!', ' ', 'h')
0.20833333333333334
(' ', 'h', 'o')
0.85
('h', 'o', 'w')
0.85
('o', 'w', ' ')
0.85
('w', ' ', 'n')
0.05
(' ', 'n', 'o')
0.25
('n', 'o', 'b')
0.25
('o', 'b', 'l')
0.25
('b', 'l', 'e')
0.625
('l', 'e', ' ')
0.125
('e', ' ', 'i')
0.15625
(' ', 'i', 'n')
0.75
('i', 'n', ' ')
0.53125
('n', ' ', 'r')
0.027777777777777776
(' ', 'r', 'e')
0.25
('r', 

In [436]:
tri_gram_add_one = np.zeros((dimension, dimension, dimension))
for w1 in range(dimension):
    for w2 in range(dimension):
        for w3 in range(dimension):
            tri_gram_add_one[w1][w2][w3] = (triGram[w1][w2][w3] + 1) / (sum(triGram[w1][w2]) + (dimension*dimension))
print(tri_gram_add_one)

[[[ 0.00111111  0.00111111  0.00111111 ...,  0.00111111  0.00111111
    0.00111111]
  [ 0.00092937  0.00092937  0.00185874 ...,  0.00092937  0.00092937
    0.00092937]
  [ 0.00081967  0.00081967  0.06803279 ...,  0.00081967  0.00081967
    0.00081967]
  ..., 
  [ 0.00109051  0.00109051  0.00109051 ...,  0.00109051  0.00109051
    0.01962923]
  [ 0.00110132  0.00110132  0.00110132 ...,  0.00110132  0.00991189
    0.00110132]
  [ 0.00111111  0.00111111  0.00111111 ...,  0.00111111  0.00111111
    0.00111111]]

 [[ 0.00097466  0.00292398  0.00877193 ...,  0.00097466  0.00097466
    0.00097466]
  [ 0.00111111  0.00111111  0.00111111 ...,  0.00111111  0.00111111
    0.00111111]
  [ 0.00110988  0.00110988  0.00110988 ...,  0.00110988  0.00110988
    0.00110988]
  ..., 
  [ 0.00110865  0.00110865  0.00110865 ...,  0.00110865  0.00110865
    0.00332594]
  [ 0.00111111  0.00111111  0.00111111 ...,  0.00111111  0.00111111
    0.00111111]
  [ 0.00111111  0.00111111  0.00111111 ...,  0.00111111  0